In [20]:
import pandas as pd

In [21]:
df=pd.read_csv("AAAI 2024/kidsInMindSubtitles2004 (2).csv")
df.shape

(2004, 15)

In [22]:
import openai

In [23]:
openai.api_key = ""

In [24]:
def generate_language_description(subtitle):
    # Language model prompt for GPT-3.5
    prompt = f"Given the list of subtitle: '{subtitle}', describe the language used in the video in 2 lines depending on quantity (more F-words, for instance, will mean a higher Language rating, and so on) as well as context (especially when it comes to the categories of sex, nudity, violence and gore, since they are not as easily quantifiable as profanity. Indicate each type of profane words for each of them."

    # Initialize conversation with the initial prompt
    conversation = [
        {"role": "system", "content": "You are a language describer."},
        {"role": "user", "content": prompt}
    ]

    # Make an API call to GPT-3.5
    response = openai.Completion.create(
        engine="text-davinci-003",
#         messages=conversation,
        prompt=prompt,
        max_tokens=500,  # Adjust this parameter to control the response length
        temperature=0.7,  # Adjust this parameter to control the response randomness
    )

    # Extract the language description from the model's response
    language_description = response.choices[0].text.strip()

    return language_description

In [30]:
def shorten_subtitle(subtitle, max_tokens):
    if len(subtitle) <= max_tokens:
        return subtitle
    
    words = subtitle.split()

    shortened_subtitle = ""

    for word in words:
        if len(shortened_subtitle) + len(word) + 1 <= max_tokens:
            shortened_subtitle += word + " "
        else:
            break

    shortened_subtitle = shortened_subtitle.strip()

    return shortened_subtitle

In [31]:
import time

In [32]:
texts = df['subtitles'].tolist()
labels_regression = df['Language'].tolist()
predictions=[]

In [41]:
for text in texts[2003:]:
    shortened_subtitle=shorten_subtitle(text, 2000)
    predictions.append(generate_language_description(shortened_subtitle))
    time.sleep(20)

In [42]:
len(predictions)

201

In [43]:
df_predictions_chatgpt=pd.DataFrame({"Predictions":predictions})

In [45]:
df_predictions_chatgpt.to_excel("summaries_Chatgpt.xlsx")

In [1]:
import pandas as pd

In [46]:
df_excel=pd.read_excel("summaries_Chatgpt.xlsx")

In [47]:
len(df_excel)

201

In [56]:
Ground_truth=texts[1803:]

In [58]:
Language_Description=df["Language_Description"].tolist()

In [51]:
len(Ground_truth)

201

In [59]:
df_predictions_chatgpt=pd.DataFrame({"Subtitles":Ground_truth,"Language_Description":Language_Description[1803:],"Predictions":df_excel['Predictions']})

In [60]:
df_predictions_chatgpt.to_excel("predictions_summarization_chatgpt.xlsx")

In [61]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


summaries = predictions
references = Ground_truth
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for summary, reference in zip(summaries, references):
    scores = scorer.score(summary, reference)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

average_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
average_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
average_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)

In [62]:
print("average_rouge1_score: ",average_rouge1_score)
print("average_rouge2_score: ",average_rouge2_score)
print("average_rougeL_score: ",average_rougeL_score)

average_rouge1_score:  0.005959579102255901
average_rouge2_score:  0.0009530633869575544
average_rougeL_score:  0.005053405809320419


In [65]:
import bert_score

model_type = 'facebook/bart-large-cnn'

P, R, F1 = bert_score.score(summaries, references, lang='en', model_type=model_type)
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)

BERTScore Precision: 0.38172194361686707
BERTScore Recall: 0.26615068316459656
BERTScore F1: 0.31311601400375366
